In [ ]:
# from google.colab import drive
import pandas as pd
import numpy as np
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
import seaborn as sns
from scipy.stats import ranksums
import pickle

from datetime import datetime
import random
from random import sample
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cluster_num=25
file_name='post_combat_gated_CD4_modify_noHD.csv'
condition_list=['*Infection','*Infection','*Infection','*Ascension','*FollowUp','*Ascension','*FollowUp','*Ascension','*FollowUp']
name_to_select=['inf01','inf02','inf12','inf1-asc01','inf1-fol01','inf2-asc01','inf2-fol01','asc01','fol01']
label_list=[[0,1],[0,2],[1,2],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]
phy_6=['143Nd_CD45RA', '167Er_CD197_CCR7','164Dy_CD95', '141Pr_CD196_CCR6', '156Gd_CD183_CXCR3', '149Sm_CD194_CCR4']
infection_select_idx=[3,4,5,6,8]
select_idx = 0# change the index number if needed-  the index matching to name_to_select condition

# TSNE color define
colors = [
"#FF0000",  # Red
"#FFA500",  # Orange
"#FFFF00",  # Yellow
"#008000",  # Green
"#00FFFF",  # Cyan
"#0000FF",  # Blue
"#FF00FF",  # Magenta
"#800080",  # Purple
"#FFC0CB",  # Pink
"#FF4500",  # Orange Red
"#FFD700",  # Gold
"#808000",  # Olive
"#008080",  # Teal
"#000080",  # Navy
"#FF1493",  # Deep Pink
"#FF69B4",  # Hot Pink
"#BA55D3",  # Medium Orchid
"#8A2BE2",  # Blue Violet
"#1E90FF",  # Dodger Blue
"#808080",  # Gray
"#4B0082",  # Indigo
"#7FFF00",  # Chartreuse
"#FFA07A",  # Light Salmon
"#800000",  # Maroon
"#20B2AA",  # Light Sea Green
]

check matrix size:  (38, 224) 38


In [ ]:
def tsne_initial(matrix_length,in_datetime_object,in_PTID,inall_cell_id,incell_label,expression_matrix):
    tsne_cell_label=[]
    sub_trans=np.empty((0,matrix_length), float)
    index_list=list()
    random.seed(in_datetime_object.timestamp())
    for single_sample in in_PTID: #1~n_clusters
        ii = np.where(np.array(inall_cell_id) == single_sample)[0]
        sampled_list = random.sample(list(ii), 1000)
        tsne_cell_label.extend(incell_label[sampled_list])
        index_list.extend(sampled_list)
        sub_trans=np.append(sub_trans, expression_matrix[sampled_list], axis=0)
    tsne = TSNE(n_components=2, perplexity=80,random_state=1024)
    in_all_embedded = tsne.fit_transform(sub_trans)
    return in_all_embedded,tsne_cell_label,index_list

def tsne_all_cluster(set_colors,tsne_df,in_tsne_path):
    sns.palplot(sns.color_palette(set_colors))

    plt.figure(figsize=(26, 26))
    sns.set(font_scale=4)
    sns.set_style(style='white')

    ax=sns.scatterplot(x='TSNE1', y='TSNE2', data=tsne_df, hue='target',palette=sns.color_palette(set_colors),legend=False, s=90)
    # for x_l in list(pca_df['target']):
    #     plt.annotate(str(x_l), pca_df.loc[pca_df['target']==str(x_l),['TSNE1','TSNE2']].mean(),horizontalalignment='center',verticalalignment='center',size=20, weight='bold')

    ax.set_xticks([])
    ax.set_yticks([])
    ax.set(xlabel='TSNE1')
    ax.set(ylabel='TSNE2')
    # plt.savefig(in_tsne_path, dpi=300)
    plt.plot()

def tsne_freq_phenotype(in_Frame,tsne_df,in_all_embedded,in_tsne_path):
    sns.set(font_scale=4)
    sns.set_style(style='white')
    fig, axes = plt.subplots(2,6, figsize=(72, 11), gridspec_kw={'height_ratios': [16,1]})
    for i in range(0,6,1):
        labels_cc=in_Frame.iloc[:,i]
        trans_temp = np.copy(tsne_df['target'])
        for singless in range(0,len(in_Frame),1):
            trans_temp[trans_temp==str(singless+1)]=labels_cc[singless]
        pca_temp = pd.DataFrame(data=in_all_embedded, columns=['TSNE1', 'TSNE2'])
        pca_temp['target']=trans_temp
        ax=sns.scatterplot(ax=axes[0,i],x='TSNE1',y='TSNE2',data=pca_temp,hue='target',palette="viridis_r",legend=False,s=3)#
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('TSNE1', fontdict=dict(weight='bold'))
        ax.set_ylabel('TSNE2', fontdict=dict(weight='bold'))
        norm = plt.Normalize(pca_temp['target'].min(), pca_temp['target'].max())
        sm = plt.cm.ScalarMappable(cmap="viridis_r", norm=norm)
        sm.set_array([])
        cbar =ax.figure.colorbar(sm,orientation='horizontal',cax=axes[1,i])
        cbar.ax.tick_params(labelsize=50)
        for t in cbar.ax.xaxis.get_major_ticks():
            t.label1.set_fontweight('bold')
        get_phy_name=np.array(in_Frame.columns)[i]
        get_phy_name=get_phy_name.split("_")
        axes[0,i].set_title(get_phy_name[len(get_phy_name)-1], fontdict = { 'weight': 'bold'})
    # plt.savefig(in_tsne_path, dpi=300)
    plt.plot()


def filter_infection(in_dfv_0,in_cell_cluster,in_select_idx):
    if in_select_idx==3 or in_select_idx==4:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] ==1].index.tolist()) ###

    elif in_select_idx==5 or in_select_idx==6:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] ==2].index.tolist()) ###

    elif in_select_idx==8:
        visit_list=np.array(in_dfv_0[in_dfv_0['*Infection'] !=2].index.tolist()) ###
    in_dfv_0=in_dfv_0.iloc[visit_list,:]
    in_cell_cluster=in_cell_cluster[visit_list]
    in_dfv_0=in_dfv_0.reset_index(drop=True)
    in_p_PTID=np.unique(in_dfv_0['*PTID'])
    return in_dfv_0,in_cell_cluster,in_p_PTID

Automated gating & tSNE visualization 

In [ ]:
#random seeds define
timestamp_string = '2023-02-03T14::12::53'
format = '%Y-%m-%dT%H::%M::%S'
datetime_object = datetime.strptime(timestamp_string, format)
random.seed(datetime_object.timestamp())


#load file
save_name=file_name.split('_')[3]
df = pd.read_csv(file_name) # path for raw data

all_cell_id=df['*PTID']
unique_ID=np.unique(df['*PTID'])
select_cell_per_ID=list()

#downsample per participate
for u_id in unique_ID:
    id_idx=np.where(np.array(all_cell_id) == u_id)[0]
    downsample_id=sample(list(id_idx), 15000)
    select_cell_per_ID.extend(downsample_id)

df=df.iloc[select_cell_per_ID]
df=df.reset_index(drop=True)
all_cell_id=df['*PTID']
unique_ID=np.unique(df['*PTID'])

get_column=df.columns

#pick frequency features
phenotype=np.array([16, 4, 3, 21, 10, 23])

features_name=[]
for i in range(0,cluster_num,1):
    features_name.append("cluster "+str(i+1)+": phenotypic markers")

pick_column=list(np.arange(0,31,1))


dfv_0_arr=np.array(df)
# arcsinh transfer
dm_0_trans = np.arcsinh(1./5 * dfv_0_arr[:,pick_column])
dm_to_df=pd.DataFrame(data=dm_0_trans,columns=get_column[pick_column])

#select functional features
for single_label in list(phenotype):
    pick_column.remove(single_label)
pick_column=np.array(pick_column)

# (1) Automated gating
# Kmean by frequency features
dm_0_trans = np.array(dm_to_df[get_column[phenotype]])
kmeans = KMeans(n_clusters=cluster_num, random_state=0).fit(dm_0_trans)
cluster_centers_arr=np.array(kmeans.cluster_centers_)
Frame=pd.DataFrame(cluster_centers_arr, columns = phy_6)
cell_label=kmeans.labels_

# plot TSNE (Figure S3A-B,D-E)
matrix_len=len(get_column[phenotype])
# TSNE initialize
all_embedded,sub_cell_label,idx_lst=tsne_initial(matrix_len,datetime_object,unique_ID,all_cell_id,cell_label,dm_0_trans)

pca_df = pd.DataFrame(data=all_embedded, columns=['TSNE1', 'TSNE2'])
pca_df['target']=[str(x+1) for x in sub_cell_label]
pca_df['index']=idx_lst

# plot TNSE for all cluster (figure 3B 3E)
tsne_plot_path= save_name + "_allcluster.png"
tsne_all_cluster(colors,pca_df,tsne_plot_path)

#plot TNSE for frequency features  (figure 3A 3D)
tsne_plot_path = save_name + "_markers_phenotype.png"
tsne_freq_phenotype(Frame,pca_df,all_embedded,tsne_plot_path)
